# Probing Many-Body Dynamics on Pulser

The following notebook shows a study of many-body dynamics on a 1D system. It is based on [1707.04344](https://arxiv.org/abs/1707.04344). Since this is a particular experiment not based on Pasqal's certified devices, we will use the `MockDevice` class to allow for a wide range of configuration settings.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip

from pulser import Pulse, Sequence, Register
from pulser.waveforms import CompositeWaveform, RampWaveform, ConstantWaveform
from pulser.devices import MockDevice
from pulser.simulation import Simulation

## Rydberg Blockade at Resonant Driving

We first consider clusters of $1, 2$ and $3$ atoms under resonant ($\delta = 0$) driving. If all the atoms are placed withing each other's blockade volume, only one excitation per group will be possible at a time. The Rabi frequency will be enhanced by $\sqrt{N}$

In [ ]:
Omega_max = 2 * 2*np.pi
R_blockade = (5.008e6/Omega_max)**(1/6)
print(f'Blockade Radius is: {R_blockade}µm.')
groups = 3

def blockade_cluster(N):
    
    atom_coords = [((R_blockade/N)*x+40*group,0) for group in range(groups) for x in range(1,N+1)]
    print(atom_coords)
    reg = Register.from_coordinates(atom_coords, prefix='q')
    
    reg.draw()
    
    resonant_pulse = Pulse.ConstantPulse(1500, Omega_max, 0., 0.)
    
    seq = Sequence(reg, MockDevice)
    seq.declare_channel('ising', 'rydberg_global')
    seq.add(resonant_pulse, 'ising')
    
    simul = Simulation(seq)
    
    up = qutip.basis(2,0)
    
    def occupation(j):
        prod = [qutip.qeye(2) for _ in range(simul._size)]
        prod[j] = up*up.dag()
        return qutip.tensor(prod)

    obs = [sum(occupation(j) for j in range(i,i+N)) for i in range(0,groups*N,N)]    
    
    simul.run(obs_list=obs, progress_bar=True)
    return simul.output.expect
    

In [ ]:
data = [blockade_cluster(N) for N in [1,2,3]]  


In [ ]:
for N,expectation in enumerate(data):
    plt.xlabel('Time (µs)')
    plt.ylabel('Probability')
    plt.title(f'N={N+1}')
    plt.plot(sum(expectation)/groups)
    plt.show()


plt.show()

Only one excitation will be shared between the atoms on each cluster. Notice how the Rabi frequency increases by a factor of $\sqrt{N}$ 

# Ordered Crystalline phases

The pulse sequence that we will prepare is based on the following parameters:

In [ ]:
# Parameters in MHz and ns
delta_0 = -4 * 2*np.pi
delta_f = 20 * 2*np.pi
Omega_max = 2 * 2*np.pi  
t_rise = 600
t_stop = 2500
t_end = 3000

We calculate the blockade radius from the maximal applied Rabi frequency:

In [ ]:
R_blockade = (5.008e6/Omega_max)**(1/6)
reg = Register.rectangle(1,11, spacing=6, prefix='q')
print(f'Blockade Radius is: {R_blockade}µm.')
reg.draw()

Create the pulses using Pulser objects:

In [ ]:
hold = ConstantWaveform(t_rise, delta_0)
excite = RampWaveform(t_stop - t_rise, delta_0, delta_f)
sweep = Pulse.ConstantAmplitude(Omega_max, CompositeWaveform(hold, excite), 0.)
stay = Pulse.ConstantPulse(t_end - t_stop, 0., delta_f, 0.)

In [ ]:
seq = Sequence(reg, MockDevice)
seq.declare_channel('ising', 'rydberg_global')

seq.add(sweep, 'ising')
seq.add(stay, 'ising')

#print(seq)
seq.draw()

## Simulation

(Running with `nsteps = 2000`, `max_step=5`)

In [ ]:
simul = Simulation(seq)
print(simul._U, Omega_max, simul._U/Omega_max)

In [ ]:
up = qutip.basis(2,0)
def occupation(j):
    prod = [qutip.qeye(2) for _ in range(simul._size)]
    prod[j] = up*up.dag()
    return qutip.tensor(prod)
    
occup_list = [occupation(j) for j in range(simul._size)]

In [ ]:
simul.run(obs_list=occup_list, progress_bar=True)
res = simul.output.expect

In [ ]:
plt.xlabel('Time (ns)')
plt.ylabel('Rydberg Occupation Probabilty')
for x in res:
    plt.plot(x)

In [ ]:
def heat_detuning(data,start,end):
    time_window = []
    x =[]
    detunings = simul.samples['Global']['ground-rydberg']['det']

    for t,d in enumerate(detunings):
        if start <= d <= end:
            time_window.append(t)
            x.append(d/(2*np.pi))

    y = np.arange(len(reg.qubits))

    X, Y = np.meshgrid(x, y)
    Z = np.array(data)[:,time_window]

    plt.figure(figsize=(20,3))
    plt.pcolormesh(X,Y,Z, cmap='hot', shading='auto')
    plt.xlabel('Detuning/2π (MHz)')
    plt.ylabel('Position in cluster')


    plt.show()

In [ ]:
heat_detuning(res,delta_0,delta_f)

# Rydberg Crystals: $Z_3$ Order

We now reduce the interatomic distance, thus increasing the interaction range between the atoms. This will lead to a $Z_3$ ordered phase: 

In [ ]:
reg = Register.rectangle(1,10, spacing=3.5, prefix='q')
reg.draw()

delta_0 = -4 * 2*np.pi
delta_f = 12 * 2*np.pi
Omega_max = 2.0 * 2*np.pi  # btw 1.8-2 Mhz*2pi
t_rise = 600
t_stop = 2500
t_end = 3000

#
hold = ConstantWaveform(t_rise, delta_0)
excite = RampWaveform(t_stop - t_rise, delta_0, delta_f)
sweep = Pulse.ConstantAmplitude(Omega_max, CompositeWaveform(hold, excite), 0.)
stay = Pulse.ConstantPulse(t_end - t_stop, 0., delta_f, 0.)

#
seq = Sequence(reg, MockDevice)
seq.declare_channel('ising', 'rydberg_global')

seq.add(sweep, 'ising')
seq.add(stay, 'ising')

#print(seq)
#seq.draw()

simul = Simulation(seq)

#
up = qutip.basis(2,0)
def occupation(j):
    prod = [qutip.qeye(2) for _ in range(simul._size)]
    prod[j] = up*up.dag()
    return qutip.tensor(prod)
    
occup_list = [occupation(j) for j in range(simul._size)]

#
simul.run(obs_list=occup_list, progress_bar=True)
res = simul.output.expect

plt.figure(figsize=(10,5))
plt.xlabel('Time (ns)')
plt.ylabel('Rydberg Occupation Probabilty')
for expv in simul.output.expect:
    plt.plot(expv)
    
heat_detuning(res,delta_0,delta_f)
    
plt.show()

# Rydberg Crystals: Z4 Order

Decreasing even more the interatomic distance leads to a $Z_4$ order. The magnitude of the Rydberg interaction with respect to that of the applied pulses means our solver has to control terms with a wider range, thus leading to longer simulation times:

In [ ]:
reg = Register.rectangle(1,9, spacing=3, prefix='q')
reg.draw()

# Parameters in MHz and ns
#U = 1 * 2*np.pi  # btw 1-3 Mhz*2pi. Should we include this VdW magnitude in an AFM-type Simulation?

delta_0 = -4 * 2*np.pi
delta_f = 10 * 2*np.pi
Omega_max = 2.0 * 2*np.pi  # btw 1.8-2 Mhz*2pi
t_rise = 600
t_stop = 2500
t_end = 3000

#
hold = ConstantWaveform(t_rise, delta_0)
excite = RampWaveform(t_stop - t_rise, delta_0, delta_f)
sweep = Pulse.ConstantAmplitude(Omega_max, CompositeWaveform(hold, excite), 0.)
stay = Pulse.ConstantPulse(t_end - t_stop, 0., delta_f, 0.)

#
seq = Sequence(reg, MockDevice)
seq.declare_channel('ising', 'rydberg_global')

seq.add(sweep, 'ising')
seq.add(stay, 'ising')

#print(seq)
#seq.draw()

simul = Simulation(seq)

#
up = qutip.basis(2,0)
def occupation(j):
    prod = [qutip.qeye(2) for _ in range(simul._size)]
    prod[j] = up*up.dag()
    return qutip.tensor(prod)
    
occup_list = [occupation(j) for j in range(simul._size)]

#
simul.run(obs_list=occup_list, progress_bar=True)
res = simul.output.expect
        
heat_detuning(res,delta_0,delta_f)
    
#
plt.figure(figsize=(10,5))
plt.xlabel('Time (ns)')
plt.ylabel('Rydberg Occupation Probabilty')
for expv in simul.output.expect:
    plt.plot(expv)
    
plt.show()